# **IRB Retail Mortgage**
## **Feature Engineering**

In this notebook, we move from **exploratory analysis** to **feature engineering**, which is a core step in retail PD model development.

Based on the outcomes of the previous notebook [`01_data_preparation_and_eda.ipynb`](../notebooks/01_data_preparation_and_eda.ipynb), we have finalised the modelling base table, defined the target variable (`default_flag`), documented variable-level treatment strategies. The focus of this notebook is:

1. Variable categorisation (numeric, categorical, count)
2. Coarse classing and binning strategy
3. WOE calculation
4. Information Value (IV) analysis
5. Variable screening and shortlist creation

Model estimation and validation are handled in subsequent notebooks.

In [ ]:
# importing the required modules
import sys
sys.path.append("..")

from src.config import *
from src.utility_functions import *

In [2]:
# loading the dataset after EDA and preliminary variable strategy decisions
model_df = save_as_pickle_if_not_exists_and_load(pickle_file_path = f"{main_dir}/data/processed/mortgage_model_data.pkl")
model_df.head(5)

Pickle file already exists at: /Users/sharmadipanshu/Developer/KPMG/irb_retail_pd_sas_vs_python//data/processed/mortgage_model_data.pkl
(2260668, 22)


,id,term,loan_amnt,int_rate,installment,grade,sub_grade,purpose,verification_status,annual_inc,emp_length,home_ownership,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,t0,default_flag
0,68407277,36 months,3600.0000,13.9900,123.0300,C,C4,debt_consolidation,Not Verified,55000.0000,10+ years,MORTGAGE,5.9100,0.0000,1.0000,7.0000,0.0000,2765.0000,29.7000,13.0000,2015-12-31,0
1,68355089,36 months,24700.0000,11.9900,820.2800,C,C1,small_business,Not Verified,65000.0000,10+ years,MORTGAGE,16.0600,1.0000,4.0000,22.0000,0.0000,21470.0000,19.2000,38.0000,2015-12-31,0
2,68341763,60 months,20000.0000,10.7800,432.6600,B,B4,home_improvement,Not Verified,63000.0000,10+ years,MORTGAGE,10.7800,0.0000,0.0000,6.0000,0.0000,7869.0000,56.2000,18.0000,2015-12-31,0
3,66310712,60 months,35000.0000,14.8500,829.9000,C,C5,debt_consolidation,Source Verified,110000.0000,10+ years,MORTGAGE,17.0600,0.0000,0.0000,13.0000,0.0000,7802.0000,11.6000,17.0000,2015-12-31,0
4,68476807,60 months,10400.0000,22.4500,289.9100,F,F1,major_purchase,Source Verified,104433.0000,3 years,MORTGAGE,25.3700,1.0000,3.0000,12.0000,0.0000,21929.0000,64.5000,35.0000,2015-12-31,0


In [17]:
# categorisation of variables into identification and reference variables, numerical variables, and categorical variables

# identification and reference variables
id_cols = ['id', 'issue_d', 't0']

# target variable
target_var = 'default_flag'

# numerical continuous variables 
num_continuous_vars = ['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'dti', 'revol_bal', 'revol_util']

# numerical count variables 
num_count_vars = ['delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'total_acc']

# categorical variables 
categorical_vars = ['term' , 'grade', 'sub_grade', 'purpose', 'verification_status', 'home_ownership', 'emp_length']

# checking the columns which are not present in the above deined lists 
undefined_vars = [col for col in model_df.columns.tolist() if col not in [target_var] + num_continuous_vars + num_count_vars + categorical_vars + id_cols]
print(f"Undefined Variables: {undefined_vars}")
    

Undefined Variables: []
